In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csgraph
import csv
import gc

import numpy as np
import scipy.sparse as spsprs
from sklearn import preprocessing

In [2]:
data_folder = './'

In [3]:
def get_ids(num_subjects=None):
    subject_IDs = np.genfromtxt(os.path.join(data_folder, 'subject_IDs.txt'), dtype=str)
    if num_subjects is not None:
        subject_IDs = subject_IDs[:num_subjects]

    return subject_IDs

In [4]:
input_df = pd.read_csv('./Phenotypic_V1_0b_preprocessed1.csv',low_memory=False)
subject_IDs = get_ids()
subject_IDs = subject_IDs.astype('int')

In [5]:
initial_data = input_df[input_df.SUB_ID.isin(subject_IDs)]


In [6]:
data_age  = initial_data.AGE_AT_SCAN
data_site = initial_data.SITE_ID
data_eye = initial_data.EYE_STATUS_AT_SCAN

In [7]:
initial_data.FIQ.isnull().sum()

28

In [8]:
for i in initial_data:
    print(i, '\t', initial_data[i].isnull().sum())

Unnamed: 0 	 0
Unnamed: 0.1 	 0
SUB_ID 	 0
X 	 0
subject 	 0
SITE_ID 	 0
FILE_ID 	 0
DX_GROUP 	 0
DSM_IV_TR 	 0
AGE_AT_SCAN 	 0
SEX 	 0
HANDEDNESS_CATEGORY 	 265
HANDEDNESS_SCORES 	 573
FIQ 	 28
VIQ 	 133
PIQ 	 115
FIQ_TEST_TYPE 	 137
VIQ_TEST_TYPE 	 221
PIQ_TEST_TYPE 	 202
ADI_R_SOCIAL_TOTAL_A 	 568
ADI_R_VERBAL_TOTAL_BV 	 568
ADI_RRB_TOTAL_C 	 568
ADI_R_ONSET_TOTAL_D 	 620
ADI_R_RSRCH_RELIABLE 	 566
ADOS_MODULE 	 485
ADOS_TOTAL 	 551
ADOS_COMM 	 571
ADOS_SOCIAL 	 571
ADOS_STEREO_BEHAV 	 596
ADOS_RSRCH_RELIABLE 	 594
ADOS_GOTHAM_SOCAFFECT 	 633
ADOS_GOTHAM_RRB 	 633
ADOS_GOTHAM_TOTAL 	 633
ADOS_GOTHAM_SEVERITY 	 633
SRS_VERSION 	 630
SRS_RAW_TOTAL 	 535
SRS_AWARENESS 	 815
SRS_COGNITION 	 815
SRS_COMMUNICATION 	 815
SRS_MOTIVATION 	 815
SRS_MANNERISMS 	 815
SCQ_TOTAL 	 741
AQ_TOTAL 	 818
COMORBIDITY 	 814
CURRENT_MED_STATUS 	 204
MEDICATION_NAME 	 754
OFF_STIMULANTS_AT_SCAN 	 771
VINELAND_RECEPTIVE_V_SCALED 	 699
VINELAND_EXPRESSIVE_V_SCALED 	 699
VINELAND_WRITTEN_V_SCALED 	 699
VINEL

In [9]:
anat = ['anat_cnr', 'anat_efc', 'anat_fber', 'anat_fwhm', 'anat_qi1', 'anat_snr']
func = ['func_efc', 'func_fber', 'func_fwhm', 'func_dvars', 'func_outlier', 'func_quality', 'func_mean_fd', 'func_num_fd', 'func_perc_fd', 'func_gsr']

data_func = initial_data[func]
data_anat = initial_data[anat]

In [10]:
data = initial_data.copy()

min_age, max_age = data.AGE_AT_SCAN.min(), data.AGE_AT_SCAN.max()
step, bins, block = 2, [min_age], min_age
while block < max_age:
    block += 2
    bins.append(block)
data.loc[:,'AGE_AT_SCAN'] = pd.cut(data.AGE_AT_SCAN, bins, right = False)

data = pd.get_dummies(data, columns=['AGE_AT_SCAN'])
data = pd.get_dummies(data, columns=['SITE_ID'])
data = pd.get_dummies(data, columns=['SEX'])

In [11]:
pheno_data = data[data.columns[data.columns.str.contains('^SITE|^AGE_AT_SCAN|^SEX')]]

In [12]:
pheno_list = list(data.columns[data.columns.str.contains('^SITE|^AGE_AT_SCAN|^SEX')])
anat_list  = anat
func_list  = func
feat_list  = pheno_list + anat_list + func_list

pheno_num = len(pheno_list)
anat_num  = len(anat)
func_num  = len(func)

ABIDE_modal_list = {'PHENO': pheno_list,
                    'ANAT' : anat_list,
                    'FUNC' : func_list}
ABIDE_modal_num  = {'PHENO': pheno_num,
                    'ANAT' : anat_num,
                    'FUNC' : func_num}

select_data = data[feat_list]

In [13]:
standard_list = anat_list + func_list

scaler        = preprocessing.StandardScaler()
standard_data = scaler.fit_transform(data[standard_list])

select_data[standard_list] = standard_data

/home/zfzhu/anaconda3/envs/SS_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/zfzhu/anaconda3/envs/SS_env/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [136]:
np.save('modal_feat_dict.npy',ABIDE_modal_list)
select_data.to_csv("processed_data_modal_three.csv", index = False)

In [162]:
feat_256_1 = pd.read_csv('./processed_standard_data_256_1.csv', low_memory=False)
pheno_256 = pd.read_csv('./processed_data_modal_three.csv', low_memory=False)

In [165]:
feat_256_1_list = list(feat_256_1.columns)[:-1]
ABIDE_modal_list['Correlation'] = feat_256_1_list
data_256_1 = pd.concat([pheno_256, feat_256_1], axis=1)

In [166]:
np.save('modal_feat_dict.npy',ABIDE_modal_list)
data_256_1.to_csv("processed_data_modal.csv", index = False)

In [172]:
data_256_1

,"AGE_AT_SCAN_[6.47, 8.47)","AGE_AT_SCAN_[8.47, 10.47)","AGE_AT_SCAN_[10.47, 12.47)","AGE_AT_SCAN_[12.47, 14.47)","AGE_AT_SCAN_[14.47, 16.47)","AGE_AT_SCAN_[16.47, 18.47)","AGE_AT_SCAN_[18.47, 20.47)","AGE_AT_SCAN_[20.47, 22.47)","AGE_AT_SCAN_[22.47, 24.47)","AGE_AT_SCAN_[24.47, 26.47)",...,247,248,249,250,251,252,253,254,255,label
0,0,0,0,0,0,0,0,0,1,0,...,0.683015,0.618513,0.523467,0.589773,0.339758,0.582218,0.440071,0.404878,0.624696,1.0
1,0,0,0,0,0,0,1,0,0,0,...,0.233801,0.308656,0.520674,0.481540,0.182549,0.230537,0.265057,0.104351,0.604107,1.0
2,0,0,0,1,0,0,0,0,0,0,...,0.404163,0.331042,0.124215,0.305749,0.247050,0.274164,0.460565,0.298192,0.748875,1.0
3,0,0,0,1,0,0,0,0,0,0,...,0.292307,0.057975,0.433875,0.128139,0.298321,0.145742,0.577506,0.314469,0.613340,1.0
4,0,0,0,0,0,1,0,0,0,0,...,0.284439,-0.000736,0.384570,0.544045,0.458481,0.432446,0.905890,0.427483,0.956445,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,0,0,0,0,0,0,0,0,0,0,...,0.170400,0.379690,0.293854,0.671053,0.440404,0.393274,0.981402,0.150351,0.836941,1.0
867,0,0,0,0,0,0,0,0,0,0,...,-0.071399,0.159802,0.299652,0.455071,0.122761,0.285891,0.705051,0.778522,0.847793,1.0
868,0,0,0,0,0,0,0,0,0,0,...,0.136041,0.108695,0.080942,-0.024082,-0.006926,-0.033737,0.377446,0.487701,0.866695,1.0
869,0,0,0,0,0,0,0,0,0,0,...,-0.067466,0.068437,-0.050140,-0.010477,0.180216,0.047201,0.229486,-0.053476,0.126005,1.0
